# Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time 

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt

from PIL import Image

print(tf.__version__)

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# This is needed to display the images.
%matplotlib inline



# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing 'PATH_TO_BE_CONFIGURED/frozen_inference_graph.pb' to point to a new .pb file.  

In [ ]:


# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = 'PATH_TO_BE_CONFIGURED/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'PATH_TO_BE_CONFIGURED/label.pbtxt'

#Number of classes
NUM_CLASSES = 1

## Load a (frozen) Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `1`, we know that this corresponds to `finger`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)



## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'PATH_TO_BE_CONFIGURED/Test_images'
#chnage X depnding on number of image you want to test
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 'X') ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        
        
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict
 

## Functions to Calculate IOU and Pixle Accuracy

Intersection Over Union (IOU) = TP/ TP+TN+FP+FN

Pixle Accuracy = TP+TN/ TP+TN+FP+FN
 

In [ ]:
def extract_classes(segm):
    cl = np.unique(segm)
    n_cl = len(cl)

    return cl, n_cl

def extract_masks(segm, cl, n_cl):
    h=480
    w=768
    masks = np.zeros((n_cl, h, w))
    for i, c in enumerate(cl):
        masks[i, :, :] = segm == c

    return masks

def union_classes(eval_segm, gt_segm):
    eval_cl, _ = extract_classes(eval_segm)
    gt_cl, _   = extract_classes(gt_segm)
    cl = np.union1d(eval_cl, gt_cl)
    n_cl = len(cl)
    return cl, n_cl
def pixel_accuracy(ground_truth, predicted_mask):
    '''
    sum_i(n_ii) / sum_i(t_i)
    '''

    classes_scores, number_of_classes = extract_classes(predicted_mask[0])
    masks_of_predictions=extract_masks(predicted_mask[0], cl, ncl)
    ground_truth_masks   = extract_masks(ground_truth, cl, ncl)

    sum_n_ii = 0
    sum_t_i  = 0

    for i, c in enumerate(cl):
        curr_eval_mask = masks_of_predictions[i, :, :]
        curr_gt_mask = ground_truth_masks[i, :, :]

        sum_n_ii += np.sum(np.logical_and(curr_eval_mask, curr_gt_mask))
        sum_t_i  += np.sum(curr_gt_mask)
 
    if (sum_t_i == 0):
        pixel_accuracy_ = 0
    else:
        pixel_accuracy_ = sum_n_ii / sum_t_i

    return pixel_accuracy_


In [ ]:
for image_path in TEST_IMAGE_PATHS:
  
  image = Image.open(image_path)
  
  
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)

  # Actual detection.

  output_dict = run_inference_for_single_image(image_np, detection_graph)

    
  # Visualization of the results of a mask prediction.
  vis_util.visualize_boxes_and_labels_on_image_array(image_np,
                                                     output_dict['detection_boxes']*0,[1, 1],[1.,0.],
                                                     category_index,instance_masks=output_dict.get('detection_masks'),
                                                     use_normalized_coordinates=True,line_thickness=1)
  
  fig = plt.figure(figsize=IMAGE_SIZE)
  ax = fig.gca()
  ax.grid(False)
  plt.imshow(image_np)
  mask=output_dict.get('detection_masks')
  
  
  plt.imshow(image_np)
  # modifying path to read annotation images 
  Annotation_path=image_path.replace("Test_images", "Ground_truth_of_test_images")
  Annotation_path=Annotation_path.replace("jpg", "png")
  ground_truth = Image.open(Annotation_path)
  ground_truth = np.reshape(np.array(ground_truth.getdata()),(480,768))
  pixel_accuracy_of_predicted_mask=pixel_accuracy(ground_truth, mask)
  IOU_of_predicted_mask=mean_IU(ground_truth, mask)

  print('pixel_accuracy of image',pixel_accuracy_of_predicted_mask)
  print('IOU of image',IOU_of_predicted_mask)

  
  
  